In [1]:
from Bio.SeqIO import parse
import glob
import os, sys
import config_readwrite as crw
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

config, cfn = crw.read(os.path.join(os.getcwd(), "config.bat2.ini"))

In [3]:
section='AJAM'
LIVERS = glob.glob(config[section]["liver"])

FQ_FILE = "/wynton/home/ahituv/fongsl/Artibeus_jamaicensis/liver/ATAC-seq/01.RawData/AJ_L_F_1/AJ_L_F_1_CKDL230012742-1A_H5TGHDSX7_L2_1.fq.gz"

# functions

## fastq -> fa 

In [8]:
def fastqToFa(FQ_FILE):
    """
    convert fastq -> fa
    
    require UCSC executable fastqToFa
    
    method
        1. make an fa file to write to
        2. run fastqToFa
        
    return
        FA_FILE (str) - resulting fasta file
    """
    
    # ucsc executable
    SRC = "/wynton/home/ahituv/fongsl/bin/ucsc_exe/fastqToFa"
    
    #1
    FA_FILE = FQ_FILE.strip(".fq.gz") + ".fa"
    
    #2
    cmd = " ".join([
                    SRC, 
                    FQ_FILE, 
                    FA_FILE
                    ])
    
    print(cmd)
    
    if os.path.exists(FA_FILE) is False:
        os.system(cmd)
        
    return FA_FILE

## fa -> bed

In [6]:
def faToBed(FA_FILE):
    """
    convert fa -> bed
    
    require Bio.SeqIO.parse
    
    method
        1. make an bed file to write to
        2. open bed file, fa_file
        3. print id, sequence to bed file
        
    return
        FA_FILE (str) - resulting fasta file
    """
    
    #1
    BED_FILE = FA_FILE.strip(".fa") + ".bed"
    
    #2
    if os.path.exists(BED_FILE) is False:
        
        print("making bed file", BED_FILE)
        
        with open(BED_FILE, "w") as bed:
            for record in parse(FA_FILE, "fasta"):

                #3
                print(record.id, 0, len(record.seq), sep="\t", file=bed)

    else:
        print("MADE bed file?", BED_FILE)
            
    return BED_FILE

# main

In [9]:
FA_FILE = fastqToFa(FQ_FILE)
BED_FILE = faToBed(FA_FILE)

/wynton/home/ahituv/fongsl/bin/ucsc_exe/fastqToFa /wynton/home/ahituv/fongsl/Artibeus_jamaicensis/liver/ATAC-seq/01.RawData/AJ_L_F_1/AJ_L_F_1_CKDL230012742-1A_H5TGHDSX7_L2_1.fq.gz /wynton/home/ahituv/fongsl/Artibeus_jamaicensis/liver/ATAC-seq/01.RawData/AJ_L_F_1/AJ_L_F_1_CKDL230012742-1A_H5TGHDSX7_L2_1.fa
MADE bed file? /wynton/home/ahituv/fongsl/Artibeus_jamaicensis/liver/ATAC-seq/01.RawData/AJ_L_F_1/AJ_L_F_1_CKDL230012742-1A_H5TGHDSX7_L2_1.bed


## liftover function

In [ ]:
def liftOver(bed_file, to_build, from_build, minmatch, chainfile):

    src = "/wynton/home/ahituv/fongsl/tools/evo/liftover_bed-wynton.py"
    cmd = " ".join([
        "python", 
           src, 
           bed_file,
           from_build, 
           to_build,
           "-m", 
            minmatch,
        "-c",
        chainfile
                    
          ])

    print(cmd)
    
    # run command in command line
    os.system(cmd)